In [1]:
from datasets import load_dataset,load_from_disk
from torch.utils.data import Dataset,DataLoader
from torch import nn

In [2]:
dataset = load_dataset('sst')


No config specified, defaulting to: sst/default
Found cached dataset sst (/home/soumodiptab/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
dataset2 =load_dataset('multi_nli')

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Dataset multi_nli downloaded and prepared to /home/soumodiptab/.cache/huggingface/datasets/multi_nli/default/0.0.0/591f72eb6263d1ab527561777936b199b714cda156d35716881158a2bd144f39. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
dataset2.save_to_disk('multi_nli.hf')

Saving the dataset (0/1 shards):   0%|          | 0/392702 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9815 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9832 [00:00<?, ? examples/s]

In [27]:
dataset2 = load_from_disk('multi_nli.hf')

In [26]:
dataset = load_from_disk('./data/sst.hf')

In [32]:
dataset_train = dataset['train']
dataset_test = dataset['test']
dataset_val = dataset['validation']

In [8]:
import torchtext
from torchtext.data.utils import get_tokenizer
import re
from cleantext import clean
import spacy

In [133]:
class DataCleaner:
    def __init__(self):
        self.tokenizer = get_tokenizer('basic_english')
        self.nlp = spacy.load('en_core_web_sm', disable = ['ner', 'tagger', 'parser'])
        self.stopwords = self.nlp.Defaults.stop_words

    def replace_dates(self,text):
        date_format_a = re.sub(r'\d{1,2}/\d{1,2}/\d{2,4}', ' <DATE> ', text)
        date_format_b = re.sub(
            r'[A-Za-z]{2,8}\s\d{1,2},?\s\d {4}', ' <DATE> ', date_format_a)
        date_format_c = re.sub(
            r'\d{2} [A-Z][a-z]{2,8} \d{4}', ' <DATE> ', date_format_b)
        return date_format_c

    def replace_hash_tags(self,text):
        return re.sub(r'(\s|^)#(\w+)', ' <HASHTAG> ', text)

    def remove_special_characters(self,text):
            # remove all special characters 
        return re.sub(r'[^A-Za-z0-9\s]', ' ', text)

    def remove_extra_spaces(self,text):
        return re.sub(r'\s{2,}', ' ', text)

    def replace_hyphenated_words(self,text):
        # replace hyphenated words with words seperated by space
        return re.sub(r'(\w+)-(\w+)', r' \1 \2 ', text)

    def clean_text(self,line):
        # line = line.strip()
        # line = re.sub(r'<|>', ' ', line)
        # line = self.replace_dates(line)
        # line = self.replace_hyphenated_words(line)
        # line = self.replace_hash_tags(line)
        # # remove < and > from the text
        # line = clean(line, no_emoji=True,
        #             no_urls=True,
        #             no_emails=True,
        #             no_phone_numbers=True,
        #             no_currency_symbols=True,           
        #             replace_with_url=" <URL> ",
        #             replace_with_email=" <EMAIL> ",
        #             replace_with_phone_number=" <PHONE> ",
        #             replace_with_currency_symbol=" <CURRENCY> ",
        #             lower=True)
        line = self.remove_special_characters(line)
        line = clean(line,no_numbers=True,
                     no_digits=True,
                     no_punct=True,
                     replace_with_number=" <NUMBER> ",
                     replace_with_digit=" ",
                     replace_with_punct="",
                     lower=True)
        line = self.remove_extra_spaces(line)
        tokens=self.tokenizer(line)
        return " ".join(tokens)
    
    def remove_stopwords(self,text):
        tokens = self.tokenizer(text)
        return " ".join([token for token in tokens if token not in self.stopwords])
    
    def lemmatize(self,text):
        doc = self.nlp(text)
        return " ".join([token.lemma_ for token in doc])
    
    def process(self,text):
        text = self.clean_text(text)
        text = self.remove_stopwords(text)
        text = self.lemmatize(text)
        return text

In [19]:
datacleaner = DataCleaner()


In [108]:
size = 0
for text in dataset_train:
    p = text['sentence'].split()
    size=max(len(p),size)
print(size)


52


In [33]:
print(dataset_train[0]["sentence"])
print(datacleaner.process(dataset_train[0]["sentence"]))

The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
rock destined st century s new conan s going splash greater arnold schwarzenegger jean claud van damme steven segal


/home/soumodiptab/.local/lib/python3.8/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [91]:
from  tqdm import tqdm
from collections import Counter
import torch
import numpy as np

In [134]:
class DataPipeline(Dataset):
    def __init__(self, filename,type,max_seq_len=50,min_freq=3,vocab=None):
        self.data,self.target = self.read_data(filename,type)
        self.max_seq_len = max_seq_len
        if vocab is None:
            self.vocab, self.ind2vocab,self.word_count = self.build_vocab(self.data,min_freq)
        else:
            self.vocab = vocab
            self.ind2vocab = {v: k for k, v in vocab.items()}
            # self.word_count = self.get_word_count(vocab,self.data)
        self.ind2vocab = {ind: word for word, ind in self.vocab.items()}
        

    def get_vocab(self):
        return self.vocab
    def word_to_ind(self,word):
        if word in self.vocab:
            return self.vocab[word]
        else:
            return 1
    def ind_to_word(self,ind):
        return self.ind2vocab[ind]
    
    def read_data(self,filename,type):
        datacleaner = DataCleaner()
        data =load_from_disk(filename)
        processed_data = []
        target = []
        for line in tqdm(data[type]):
            processed_data.append(datacleaner.process(line['sentence']).split(" "))
            target.append(line['label'])
        return processed_data,target
    
    def get_word_count(self,vocab,data):
        word_count = {0: 0}
        for line in data:
            for word in line:
                if word in vocab:
                    word_count[vocab[word]] += 1
                else:
                    word_count[0] += 1
        return word_count
    
    def most_common(self,n):
        counter = Counter(self.word_count)
        common = counter.most_common(n)
        ind_freq = dict(common)
        # convert to word frequency
        word_freq = {}
        for ind in ind_freq:
            word_freq[self.ind2vocab[ind]] = ind_freq[ind]
        return word_freq
    
    @staticmethod
    def build_vocab(data,min_freq):
        word_set = {}
        print('Building vocab:')
        for line in tqdm(data):
            for word in line:
                if word not in word_set:
                    word_set[word]=1
                else:
                    word_set[word]+=1
        # sort the vocab
        word_list = sorted(list(word_set))
        word_count = {0: 0, 1: 0}
        vocab_dict = {"<pad>":0,"<unk>": 1}
        i=2
        for word in tqdm(word_list):
            if word_set[word] >= min_freq:
                vocab_dict[word] = i
                word_count[i] = word_set[word]
                i+=1
            else:
                word_count[0] += word_set[word]
        ind2word = {v: k for k, v in vocab_dict.items()}
        print('Vocab size: {}'.format(len(vocab_dict)))
        return vocab_dict, ind2word, word_count

    def total_count(self):
        return sum(self.word_count.values())

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sent = self.data[idx]
        label = self.target[idx]
        # paddding the sentences to create sequences of same length
        if len(sent) < self.max_seq_len:
            sent=[self.word_to_ind(token) for token in sent]+[self.word_to_ind("<pad>") for _ in range(self.max_seq_len - len(sent))]
        return torch.LongTensor(sent),torch.Tensor([label])
    
    def get_batches(self, batch_size):
        return DataLoader(self, batch_size=batch_size, shuffle=False,drop_last=True)

In [143]:
class ElmoDataset(DataPipeline):
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        sent = self.data[idx]
        label = self.target[idx]
        # paddding the sentences to create sequences of same length
        if len(sent) < self.max_seq_len:
            sent=[self.word_to_ind(token) for token in sent]+[self.word_to_ind("<pad>") for _ in range(self.max_seq_len - len(sent))]
        forward_data = sent[1:]
        backward_data = sent[:-1]
        return torch.LongTensor(forward_data),torch.LongTensor(backward_data)

In [144]:
data_train = ElmoDataset('data/sst.hf','train',80,1)

100%|██████████| 8544/8544 [00:21<00:00, 389.37it/s]


Building vocab:


100%|██████████| 14886/14886 [00:00<00:00, 677787.29it/s]


Vocab size: 14888


In [137]:
data_validation = ElmoDataset('data/sst.hf','validation',80,1,data_train.get_vocab())

100%|██████████| 1101/1101 [00:03<00:00, 359.17it/s]


In [142]:
data_train.most_common(10)

{'s': 2612,
 'film': 1166,
 'movie': 1016,
 't': 705,
 'n': 695,
 'like': 516,
 'number': 408,
 '<': 391,
 '>': 391,
 'story': 355}

In [93]:
def load_embeddings(vocab,embeddings_file,dimension):
    # load only the embeddings that are in the vocab
    embeddings = np.zeros((len(vocab), dimension))
    with open(embeddings_file, 'r') as f:
        for line in tqdm(f):
            line = line.split()
            word = line[0]
            if word in vocab:
                embeddings[vocab[word]] = np.array(line[1:], dtype=np.float32)
    return torch.Tensor(embeddings)

In [145]:
embedding_matix=load_embeddings(data_train.get_vocab(),'embeddings/glove/glove.6B.100d.txt',100)

400000it [00:04, 98159.26it/s] 


In [101]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [128]:
class ELMo(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, max_len, embedding_matrix):
        super(ELMo, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.max_len = max_len
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix)
        self.embedding.weight = nn.Parameter(self.embedding.weight, requires_grad=True)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(hidden_dim*2, hidden_dim, batch_first=True, bidirectional=True)
        self.linear_out = nn.Linear(hidden_dim*2, vocab_size)
    def forward(self,back_data):
        back_embed = self.embedding(back_data)
        back_lstm1, _ = self.lstm1(back_embed)
        #back_lstm1 is shape of (batch_size, max_len, hidden_dim*2)
        back_lstm2, _ = self.lstm2(back_lstm1)
        linear_out = self.linear_out(back_lstm2)
        return linear_out

In [127]:
elmo = ELMo(len(data_train.get_vocab()),100,100,80,embedding_matix)

In [163]:
print(elmo)

ELMo(
  (embedding): Embedding(14876, 100)
  (lstm1): LSTM(100, 100, batch_first=True, bidirectional=True)
  (lstm2): LSTM(200, 100, batch_first=True, bidirectional=True)
  (linear1): Linear(in_features=100, out_features=100, bias=True)
  (linear_out): Linear(in_features=200, out_features=14876, bias=True)
)


In [170]:
class ElmoTrainer:
    def __init__(self,epochs=20,lr=0.001,batch_size=50,print_every=1,device='cpu'):
        self.epochs = epochs
        self.lr = lr
        self.batch_size = batch_size
        self.print_every = print_every
        self.device = device
        self.criterion = nn.CrossEntropyLoss()
        self.lowest_validation_loss = float('inf')
    
    def train(self,model : ELMo,model_save_path,train_data,validation_data):
        self.optimizer = optim.Adam(model.parameters(), lr=self.lr)
        model.to(self.device)
        for epoch in range(len(range(self.epochs))):
            model.train()
            train_loader = train_data.get_batches(self.batch_size)
            training_loss = 0
            for i,(forward_data,backward_data) in tqdm(enumerate(train_loader)):
                forward_data = forward_data.to(self.device)
                backward_data = backward_data.to(self.device)
                self.optimizer.zero_grad()
                output = model(backward_data)
                output = output.view(-1, model.vocab_size)
                target = forward_data.view(-1)
                loss = self.criterion(output, target)
                loss.backward()
                self.optimizer.step()
                training_loss += loss.item()
        if epoch % self.print_every == 0:
            print('Training Loss : {}'.format(training_loss/len(train_loader)))
        self.__validate(model,model_save_path,validation_data)

    def __validate(self,model:ELMo,model_save_path,validation_data):
        model.eval()
        model.to(self.device)
        validation_loader = validation_data.get_batches(self.batch_size)
        validation_loss = 0
        for i,(forward_data,backward_data) in tqdm(enumerate(validation_loader)):
            forward_data = forward_data.to(self.device)
            backward_data = backward_data.to(self.device)
            output = model(backward_data)
            output = output.view(-1, len(model.vocab_size))
            target = forward_data.view(-1)
            loss = self.criterion(output, target)
            validation_loss += loss.item()
        if validation_loss < self.lowest_validation_loss:
            self.lowest_validation_loss = validation_loss
            torch.save(model.state_dict(), model_save_path)
        print('Validation Loss : {}'.format(validation_loss/len(validation_loader)))






In [171]:
trainer = ElmoTrainer(epochs=20,lr=0.001,batch_size=50,print_every=2,device='cpu')

In [172]:
trainer.train(elmo,'model/elmo.pt',data_train,data_validation)

0it [00:00, ?it/s]

: 

: 